In [1]:
import os

In [2]:
%pwd

'c:\\Users\\08you\\Desktop\\My own 3 hour project\\Securing-Medical-Data-Transfer-Using-Deep-Steganography-and-Cryptography-MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\08you\\Desktop\\My own 3 hour project\\Securing-Medical-Data-Transfer-Using-Deep-Steganography-and-Cryptography-MLflow-DVC'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]= "https://dagshub.com/08Youssef08/Securing-Medical-Data-Transfer-Using-Deep-Steganography-and-Cryptography-MLflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]= "08Youssef08"
os.environ["MLFLOW_TRACKING_PASSWORD"]= "693be62f3a1b740d8a2ea81a6ad306d4e527d3e1"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model(r".\artifacts\training\model.h5",custom_objects={'custom_loss_2': custom_loss_2,'custom_loss_1': custom_loss_1})


KeyboardInterrupt



In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    Med_test: Path
    Cover_test:Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories,save_json
import tensorflow as tf

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= r"artifacts\training\model.h5",
            Med_test= r"artifacts\data_ingestion\Data-CS-DVC\test_data\Med_test",
            Cover_test= r"artifacts\data_ingestion\Data-CS-DVC\test_data\Cover_test",
            all_params= self.params,
            mlflow_uri= r"https://dagshub.com/08Youssef08/Securing-Medical-Data-Transfer-Using-Deep-Steganography-and-Cryptography-MLflow-DVC.mlflow",
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )
        return eval_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from src.cnnClassifier.utils.common import custom_loss_1,custom_loss_2,gray_to_rgb,normalize_batch,denormalize_batch
import glob


In [19]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.TEST_NUM=len(glob.glob(str(config.Med_test)+"/*/*"))

    def generate_generator_multiple(self,generator, med_path, cover_path):
            genX1 = generator.flow_from_directory(med_path, target_size=(224, 224), batch_size=self.config.params_batch_size, shuffle=True, class_mode=None)
            genX2 = generator.flow_from_directory(cover_path, target_size=(224, 224), batch_size=self.config.params_batch_size, shuffle=True, class_mode=None)

            while True:
                X1i = normalize_batch(genX1.next())
                X2i = normalize_batch(genX2.next())

                # Check if the images are grayscale, and convert to RGB if necessary
                if X1i.shape[-1] != 3:
                    X1i = gray_to_rgb(X1i)
                if X2i.shape[-1] != 3:
                    X2i = gray_to_rgb(X2i)

                yield ({'secret': X1i, 'cover': X2i}, {'hide_conv_f': X2i, 'revl_conv_f': X1i})

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = self.generate_generator_multiple(generator=valid_datagenerator, med_path=self.config.Med_test,cover_path=self.config.Cover_test)
    

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path,custom_objects={'custom_loss_2': custom_loss_2,'custom_loss_1': custom_loss_1})
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator,steps=self.TEST_NUM/self.config.params_batch_size, verbose=1)
        self.save_score()
    def save_score(self):
        scores = {"Total loss": self.score[0],"hide_conv_f_loss":self.score[1],"revl_conv_f_loss":self.score[2]}
        save_json(path=Path("scores.json"),data=scores)
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"Total loss": self.score[0],"hide_conv_f_loss":self.score[1],"revl_conv_f_loss":self.score[2]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="DeepStego1")
            else:
                mlflow.keras.log_model(self.model, "model")

In [20]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-02-12 19:56:10,564: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-12 19:56:10,569: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-12 19:56:10,571: INFO: common: created directory at: artifacts]
Found 16 images belonging to 1 classes.
Found 16 images belonging to 1 classes.
1/1 [==============================] - 26s 26s/step - loss: 2.6482 - hide_conv_f_loss: 1.6403 - revl_conv_f_loss: 1.3438
[2024-02-12 19:56:38,132: INFO: common: json file saved at: scores.json]


2024/02/12 19:56:39 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-02-12 19:56:44,632: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 48). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\08you\AppData\Local\Temp\tmpl9piyc62\model\data\model\assets
[2024-02-12 19:56:47,777: INFO: builder_impl: Assets written to: C:\Users\08you\AppData\Local\Temp\tmpl9piyc62\model\data\model\assets]


2024/02/12 19:57:02 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\08you\AppData\Local\Temp\tmpl9piyc62\model, flavor: tensorflow), fall back to return ['tensorflow==2.12.0']. Set logging level to DEBUG to see the full traceback.
c:\Users\08you\anaconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'DeepStego1'.
2024/02/12 19:57:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: DeepStego1, version 1
Created version '1' of model 'DeepStego1'.
